# COMP 7310 Personal Project

### 1. Config Setup

In [20]:
# Make Dirs for Model Storage and Test Results
import os

try:
    # Model Storage Path
    model_prediction_path = '/kaggle/working/prediction_model'
    os.mkdir(model_prediction_path)
except:
    pass
try:
    # Test Results Path
    test_result_path = '/kaggle/working/test_results'
    os.mkdir(test_result_path)
except:
    pass
try:
    # Test Seen & Unseen Path
    os.mkdir(test_result_path + '/test_seen')
    os.mkdir(test_result_path + '/test_unseen')
except:
    pass

In [21]:
"""
config settings:
"""
### TFNET parameters
BATCH_SIZE = 72 # Training batch size
TEST_BATCH_SIZE = 1 # Test batch size
EPOCHS = 150 # Traning epoch
SAVE_INTERVAL = 20 # Save model every 20 epochs
STEP_SIZE = 200 # Step size for moving forward the window (For training)
TEST_STEP_SIZE = 400 # Step size for moving forward the window (For testing)
WINDOW_SIZE = 400 # Window size for training and testing
INPUT_CHANNEL = 6 # Input feature dimension (Gryo + Acce)
OUTPUT_CHANNEL = 2 # Output dimension (2D velocity vector)
SAMPLING_RATE = 200 # Sampling rate
LAYER_SIZE = 100 # The size of LSTM
LAYERS = 3 # The layer size of LSTM
DROPOUT = 0.1 # Dropout probability
LEARNING_RATE = 0.0003 # Learning rate
NUM_WORKERS = 8
### ------------------ ###

### Data preprocessing parameters
FEATURE_SIGMA = 2.0 # Sigma for feature gaussian smoothing
TARGET_SIGMA = 30.0 # Sigma for target gaussian smoothing
### ------------------ ###

### Device for training
DEVICE = "cuda:0" # You can choose GPU or CPU
### ------------------ ###

### Training and testing setting
DATA_DIR = '/kaggle/input/comp7310-project-1-imu-indoor-tracking/original_data/train_dataset' # Dataset directory for training
VAL_DATA_DIR = '/kaggle/input/comp7310-project-1-imu-indoor-tracking/original_data/val_dataset' # Dataset directory for validation
TEST_DIR = '/kaggle/input/comp7310-project-1-imu-indoor-tracking/original_data/test_seen' # Dataset directory for testing (unseen_subjects_test_set)
OUT_DIR = '/kaggle/working/prediction_model' # Output directory for both traning and testing
MODEL_PATH = '' # Model path for testing
### ------------------ ###

def load_config():
    kwargs = {}
    kwargs['batch_size'] = BATCH_SIZE
    kwargs['test_batch_size'] = TEST_BATCH_SIZE
    kwargs['epochs'] = EPOCHS
    kwargs['save_interval'] = SAVE_INTERVAL
    kwargs['step_size'] = STEP_SIZE
    kwargs['test_step_size'] = TEST_STEP_SIZE
    kwargs['window_size'] = WINDOW_SIZE
    kwargs['sampling_rate'] = SAMPLING_RATE
    kwargs['input_channel'] = INPUT_CHANNEL
    kwargs['output_channel'] = OUTPUT_CHANNEL
    kwargs['layer_size'] = LAYER_SIZE
    kwargs['layers'] = LAYERS
    kwargs['dropout'] = DROPOUT
    kwargs['learning_rate'] = LEARNING_RATE
    kwargs['num_workers'] = NUM_WORKERS

    kwargs['feature_sigma'] = FEATURE_SIGMA
    kwargs['target_sigma'] = TARGET_SIGMA

    kwargs['device'] = DEVICE

    kwargs['data_dir'] = DATA_DIR
    kwargs['val_data_dir'] = VAL_DATA_DIR
    kwargs['test_dir'] = TEST_DIR
    kwargs['out_dir'] = OUT_DIR
    kwargs['model_path'] = MODEL_PATH

    return kwargs


### 2. Model Design

In [22]:
import torch
from scipy import signal
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch.nn.utils import weight_norm

### BiLSTM model
class BilinearLSTMSeqNetwork(torch.nn.Module):
    def __init__(self, input_size, out_size, batch_size, device,
                 lstm_size=100, lstm_layers=3, dropout=0):
        """
        LSTM network with Bilinear layer
        Input: torch array [batch x frames x input_size]
        Output: torch array [batch x frames x out_size]
        :param input_size: num. channels in input
        :param out_size: num. channels in output
        :param batch_size:
        :param device: torch device
        :param lstm_size: number of LSTM units per layer
        :param lstm_layers: number of LSTM layers
        :param dropout: dropout probability of LSTM (@ref https://pytorch.org/docs/stable/nn.html#lstm)
        """
        super(BilinearLSTMSeqNetwork, self).__init__()
        self.input_size = input_size
        self.lstm_size = lstm_size
        self.output_size = out_size
        self.num_layers = lstm_layers
        self.batch_size = batch_size
        self.device = device

        self.bilinear = torch.nn.Bilinear(self.input_size, self.input_size, self.input_size * 4)
        self.lstm = torch.nn.LSTM(self.input_size * 5, self.lstm_size, self.num_layers, batch_first=True, dropout=dropout)
        self.linear1 = torch.nn.Linear(self.lstm_size + self.input_size * 5, self.output_size * 5)
        self.linear2 = torch.nn.Linear(self.output_size * 5, self.output_size)
        self.hidden = self.init_weights()

    def forward(self, input):
        input_mix = self.bilinear(input, input)
        input_mix = torch.cat([input, input_mix], dim=2)
        output, self.hidden = self.lstm(input_mix, self.init_weights())
        output = torch.cat([input_mix, output], dim=2)
        output = self.linear1(output)
        output = self.linear2(output)
        return output

    def init_weights(self):
        h0 = torch.zeros(self.num_layers, self.batch_size, self.lstm_size)
        c0 = torch.zeros(self.num_layers, self.batch_size, self.lstm_size)
        h0 = h0.to(self.device)
        c0 = c0.to(self.device)
        return Variable(h0), Variable(c0)
### --------------------- End of the model --------------------- ###

### 3. Data Loader

In [23]:
!pip install pyquaternion==0.9.9
!pip install numpy-quaternion==2022.4.3

In [24]:
import json
import random
import os
from os import path as osp

import h5py
import torch
import numpy as np
import quaternion
import math
from scipy.ndimage import gaussian_filter1d
from torch.utils.data import Dataset

def convert_data(data_path):
    """
    Data Processing
    :param data_path
    This function is used to convert the raw data
    stored in hdf5 format to numpy array
    """
    # read hdf5 file
    file = h5py.File(os.path.join(data_path, 'data.hdf5'), 'r')
    synced = file['synced'] # synced data (gyro, acce, magn ...)
    pose = file['pose'] # pose data (ground truth)
    # timestamp (1D)
    timestamps = np.array(synced['time'])
    timestamps = timestamps.reshape((len(timestamps), 1)) / 10**9 # - timestamps[0] # start from 0
    timestamps = timestamps - timestamps[0]
    # gyroscope (3D), accelerometer (3D), magnetometer (3D), rotation vector (4D)
    gyro = np.array(synced['gyro'])
    acce = np.array(synced['acce'])
    # game rotation vector can convert acce and gyro
    # from body frame to navigation frame
    rotation_vector = synced['game_rv']

    # position (3D), orientation (4D)
    pose = file['pose']
    tango_pos = np.array(pose['tango_pos'])
    tango_ori = np.array(pose['tango_ori'])

    # Compute the IMU orientation in the Tango coordinate frame.
    init_tango_ori = quaternion.quaternion(*tango_ori[0])
    ori = rotation_vector
    ori_q = quaternion.from_float_array(ori)
    init_rotor = init_tango_ori * ori_q[0].conj()
    ori_q = init_rotor * ori_q

    gyro_q = quaternion.from_float_array(np.concatenate([np.zeros([gyro.shape[0], 1]), gyro], axis=1))
    acce_q = quaternion.from_float_array(np.concatenate([np.zeros([acce.shape[0], 1]), acce], axis=1))
    glob_gyro = quaternion.as_float_array(ori_q * gyro_q * ori_q.conj())[:, 1:]
    glob_acce = quaternion.as_float_array(ori_q * acce_q * ori_q.conj())[:, 1:]

    rawdata = np.concatenate((timestamps, glob_gyro, glob_acce), axis = 1)
    groundtruth = np.concatenate((timestamps, tango_pos, tango_ori), axis = 1)

    return rawdata, groundtruth

def convert_data_test(data_path):
    """
    Data Processing
    :param data_path
    This function is used to convert the raw data
    stored in hdf5 format to numpy array
    """
    # read hdf5 file
    file = h5py.File(os.path.join(data_path, 'data.hdf5'), 'r')
    synced = file['synced'] # synced data (gyro, acce, magn ...)
    pose = file['pose'] # pose data (ground truth)
    # timestamp (1D)
    timestamps = np.array(synced['time'])
    timestamps = timestamps.reshape((len(timestamps), 1)) / 10**9 # - timestamps[0] # start from 0
    timestamps = timestamps - timestamps[0]
    # gyroscope (3D), accelerometer (3D), magnetometer (3D), rotation vector (4D)
    gyro = np.array(synced['gyro'])
    acce = np.array(synced['acce'])
    # game rotation vector can convert acce and gyro
    # from body frame to navigation frame
    rotation_vector = synced['game_rv']

    # position (3D), orientation (4D)
    pose = file['pose']
    tango_ori = np.array(pose['tango_ori'])

    # Compute the IMU orientation in the Tango coordinate frame.
    init_tango_ori = quaternion.quaternion(*tango_ori[0])
    ori = rotation_vector
    ori_q = quaternion.from_float_array(ori)
    init_rotor = init_tango_ori * ori_q[0].conj()
    ori_q = init_rotor * ori_q

    gyro_q = quaternion.from_float_array(np.concatenate([np.zeros([gyro.shape[0], 1]), gyro], axis=1))
    acce_q = quaternion.from_float_array(np.concatenate([np.zeros([acce.shape[0], 1]), acce], axis=1))
    glob_gyro = quaternion.as_float_array(ori_q * gyro_q * ori_q.conj())[:, 1:]
    glob_acce = quaternion.as_float_array(ori_q * acce_q * ori_q.conj())[:, 1:]

    rawdata = np.concatenate((timestamps, glob_gyro, glob_acce), axis = 1)

    return rawdata

class GlobSequence():
    """
    Property: global coordinate frame
    """
    # add 3-axis magnetometer
    # feature_dim = 9
    feature_dim = 6
    target_dim = 2
    aux_dim = 8

    def __init__(self, data_path = None, **kwargs):
        super().__init__()
        self.ts, self.features, self.targets, self.gt_pos = None, None, None, None
        # self.info = {}
        self.w = kwargs.get('interval', 1)
        if data_path is not None:
            self.load(data_path)

    def load(self, data_path):
        # print("the data_path is:", data_path)
        data, ground_truth = convert_data(data_path)
        # already in global coordinate frame and start from start frame
        # timestamp (1D) gyroscope (3D), accelerometer (3D)
        gyro = data[:, 1:4]
        acce = data[:, 4:7]
        ts = data[:, 0]
        # tango position
        tango_pos = ground_truth[:, 1:4]
        # tango orientation
        tango_ori = ground_truth[:, 4:8]

        dt = (ts[self.w:] - ts[:-self.w])[:, None]
        # calculate the global velocity
        glob_v = (tango_pos[self.w:] - tango_pos[:-self.w]) / dt

        self.ts = ts
        self.features = np.concatenate([gyro, acce], axis = 1)
        # We only use the global velocity in the floor plane
        self.targets = glob_v[:, :2]
        self.orientations = tango_ori # quaternion.as_float_array(tango_ori)
        self.gt_pos = tango_pos

    def get_feature(self):
        return self.features

    def get_target(self):
        return self.targets

    def get_aux(self):
        return np.concatenate([self.ts[:, None], self.orientations, self.gt_pos], axis = 1)

class GlobSequenceTest():
    """
    Property: global coordinate frame
    """
    # add 3-axis magnetometer
    # feature_dim = 9
    feature_dim = 6
    aux_dim = 8

    def __init__(self, data_path = None, **kwargs):
        super().__init__()
        self.ts, self.features, self.targets, self.gt_pos = None, None, None, None
        # self.info = {}
        self.w = kwargs.get('interval', 1)
        if data_path is not None:
            self.load(data_path)

    def load(self, data_path):
        # print("the data_path is:", data_path)
        data = convert_data_test(data_path)
        # already in global coordinate frame and start from start frame
        # timestamp (1D) gyroscope (3D), accelerometer (3D)
        gyro = data[:, 1:4]
        acce = data[:, 4:7]
        ts = data[:, 0]

        dt = (ts[self.w:] - ts[:-self.w])[:, None]

        self.ts = ts
        self.features = np.concatenate([gyro, acce], axis = 1)

    def get_feature(self):
        return self.features

    def get_aux(self):
        return 0

def load_sequences(seq_type, root_dir, data_list, **kwargs):
    features_all, targets_all, aux_all = [], [], []

    for i in range(len(data_list)):
        seq = seq_type(osp.join(root_dir, data_list[i]), **kwargs)
        feat, targ, aux = seq.get_feature(), seq.get_target(), seq.get_aux()
        # add feat, targ, aux to list
        features_all.append(feat)
        targets_all.append(targ)
        aux_all.append(aux)
    return features_all, targets_all, aux_all

def load_sequences_test(seq_type, root_dir, data_list, **kwargs):
    features_all, aux_all = [], []

    for i in range(len(data_list)):
        seq = seq_type(osp.join(root_dir, data_list[i]), **kwargs)
        feat, aux = seq.get_feature(), seq.get_aux()
        # add feat, targ, aux to list
        features_all.append(feat)
        aux_all.append(aux)
    return features_all, aux_all

class SequenceToSequenceDataset(Dataset):
    def __init__(self, seq_type, root_dir, data_list, step_size = 100, window_size = 400,
                 random_shift = 0, transform = None, **kwargs):
        super(SequenceToSequenceDataset, self).__init__()
        self.seq_type = seq_type
        self.feature_dim = seq_type.feature_dim
        self.target_dim = seq_type.target_dim
        self.aux_dim = seq_type.aux_dim
        self.window_size = window_size
        self.step_size = step_size
        self.random_shift = random_shift
        self.transform = transform
        self.projection_width = kwargs.get('projection_width', 0)

        self.data_path = [osp.join(root_dir, data) for data in data_list]
        self.index_map = []

        self.features, self.targets, aux = load_sequences(
            seq_type, root_dir, data_list, **kwargs)

        # Optionally smooth the sequence
        feat_sigma = kwargs.get('feature_sigma,', -1)
        targ_sigma = kwargs.get('target_sigma,', -1)
        if feat_sigma > 0:
            self.features = [gaussian_filter1d(feat, sigma=feat_sigma, axis=0) for feat in self.features]
        if targ_sigma > 0:
            self.targets = [gaussian_filter1d(targ, sigma=targ_sigma, axis=0) for targ in self.targets]

        max_norm = 3.0 #
        self.ts, self.orientations, self.gt_pos, self.local_v = [], [], [], []
        for i in range(len(data_list)):
            self.features[i] = self.features[i][:-1]
            self.targets[i] = self.targets[i]
            self.ts.append(aux[i][:-1, :1])
            self.orientations.append(aux[i][:-1, 1:5])
            self.gt_pos.append(aux[i][:-1, 5:8])

            velocity = np.linalg.norm(self.targets[i], axis=1)  # Remove outlier ground truth data
            bad_data = velocity > max_norm
            for j in range(window_size + random_shift, self.targets[i].shape[0], step_size):
                if not bad_data[j - window_size - random_shift:j + random_shift].any():
                    self.index_map.append([i, j])

        # if shuffle is necessary here? As the training data should be in a logical sequence
        if kwargs.get('shuffle', True):
            random.shuffle(self.index_map)

    def __getitem__(self, item):
        # output format: input, target, seq_id, frame_id
        seq_id, frame_id = self.index_map[item][0], self.index_map[item][1]

        feat = np.copy(self.features[seq_id][frame_id - self.window_size:frame_id])
        targ = np.copy(self.targets[seq_id][frame_id - self.window_size:frame_id])
        # random rotate the sequence in the horizontal plane
        if self.transform is not None:
            feat, targ = self.transform(feat, targ)

            return feat.astype(np.float32), targ.astype(np.float32), seq_id, frame_id

    def __len__(self):
        return len(self.index_map)

    def get_lstm_test_seq(self):
        return np.array(self.features).astype(np.float32), np.array(self.targets).astype(np.float32)

class SequenceToSequenceDatasetTest(Dataset):
    def __init__(self, seq_type, root_dir, data_list, step_size = 100, window_size = 400,
                 random_shift = 0, transform = None, **kwargs):
        super(SequenceToSequenceDatasetTest, self).__init__()
        self.seq_type = seq_type
        self.feature_dim = seq_type.feature_dim
        self.aux_dim = seq_type.aux_dim
        self.window_size = window_size
        self.step_size = step_size
        self.random_shift = random_shift
        self.transform = transform
        self.projection_width = kwargs.get('projection_width', 0)

        self.data_path = [osp.join(root_dir, data) for data in data_list]
        self.index_map = []

        self.features, aux = load_sequences_test(
            seq_type, root_dir, data_list, **kwargs)

        # Optionally smooth the sequence
        feat_sigma = kwargs.get('feature_sigma,', -1)
        if feat_sigma > 0:
            self.features = [gaussian_filter1d(feat, sigma=feat_sigma, axis=0) for feat in self.features]

        max_norm = 3.0 #
        for i in range(len(data_list)):
            self.features[i] = self.features[i][:-1]

        # if shuffle is necessary here? As the training data should be in a logical sequence
        if kwargs.get('shuffle', True):
            random.shuffle(self.index_map)

    def __getitem__(self, item):
        # output format: input, target, seq_id, frame_id
        seq_id, frame_id = self.index_map[item][0], self.index_map[item][1]
        feat = np.copy(self.features[seq_id][frame_id - self.window_size:frame_id])

        return feat.astype(np.float32), seq_id, frame_id

    def __len__(self):
        return len(self.index_map)

    def get_lstm_test_seq(self):
        return np.array(self.features).astype(np.float32)

def change_cf(ori, vectors):
    """
    Euler-Rodrigous formula v'=v+2s(rxv)+2rx(rxv)
    :param ori: quaternion [n]x4
    :param vectors: vector nx3
    :return: rotated vector nx3
    """
    assert ori.shape[-1] == 4
    assert vectors.shape[-1] == 3

    if len(ori.shape) == 1:
        ori = ori.reshape(1, -1)

    q_s = ori[:, :1]
    q_r = ori[:, 1:]

    tmp = np.cross(q_r, vectors)
    vectors = np.add(np.add(vectors, np.multiply(2, np.multiply(q_s, tmp))), np.multiply(2, np.cross(q_r, tmp)))
    return vectors

class ComposeTransform:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, feat, targ, **kwargs):
        for t in self.transforms:
            feat, targ = t(feat, targ)
        return feat, targ

class RandomHoriRotateSeq:
    def __init__(self, input_format, output_format=None):
        """
        Rotate global input, global output by a random angle
        @:param input format - input feature vector(x,3) boundaries as array (E.g [0,3,6])
        @:param output format - output feature vector(x,2/3) boundaries as array (E.g [0,2,5])
                                if 2, 0 is appended as z.
        """
        self.i_f = input_format
        self.o_f = output_format

    def __call__(self, feature, target):
        a = np.random.random() * 2 * np.math.pi
        # print("Rotating by {} degrees", a/np.math.pi * 180)
        t = np.array([np.cos(a), 0, 0, np.sin(a)])

        for i in range(len(self.i_f) - 1):
            feature[:, self.i_f[i]: self.i_f[i + 1]] = \
                change_cf(t, feature[:, self.i_f[i]: self.i_f[i + 1]])

        for i in range(len(self.o_f) - 1):
            if self.o_f[i + 1] - self.o_f[i] == 3:
                # vector = target[:, self.o_f[i]: self.o_f[i + 1]]
                # target[:, self.o_f[i]: self.o_f[i + 1]] = change_cf(t, vector)
                vector = target[self.o_f[i]: self.o_f[i + 1]]
                target[:, self.o_f[i]: self.o_f[i + 1]] = change_cf(t, vector)
            elif self.o_f[i + 1] - self.o_f[i] == 2:
                vector = np.concatenate([target[:, self.o_f[i]: self.o_f[i + 1]], np.zeros([target.shape[0], 1])], axis=1)
                target[:, self.o_f[i]: self.o_f[i + 1]] = change_cf(t, vector)[:, :2]

        return feature.astype(np.float32), target.astype(np.float32)

class RandomHoriRotateSeqTensor:
    def __init__(self):
        """
        Rotate global input, global output by a random angle
        @:param input format - input feature vector(x,3) boundaries as array (E.g [0,3,6])
        @:param output format - output feature vector(x,2/3) boundaries as array (E.g [0,2,5])
                                if 2, 0 is appended as z.
        """

    def __call__(self, feature, target):
        # Tensor random rotation matrix
        a = torch.rand(1) * 2 * np.math.pi
        rotation_matrix_feat = torch.tensor([[torch.cos(a), torch.sin(a), 0, 0, 0, 0],
                                            [-torch.sin(a), torch.cos(a), 0, 0, 0, 0],
                                            [0, 0, 1, 0, 0, 0],
                                            [0, 0, 0, torch.cos(a), torch.sin(a), 0],
                                            [0, 0, 0, -torch.sin(a), torch.cos(a), 0],
                                            [0, 0, 0, 0, 0, 1]], dtype=torch.float32)

        rotation_matrix_targ = torch.tensor([[torch.cos(a), torch.sin(a)],
                                            [-torch.sin(a), torch.cos(a)]], dtype=torch.float32)

        # Matrix multiplication
        feature = torch.matmul(feature, rotation_matrix_feat)
        target = torch.matmul(target, rotation_matrix_targ)

        return feature, target

def get_dataset(root_dir, data_list, mode, **kwargs):
    # load config
    global_step_size = 0
    # input data includes: accelemeters, gyroscopes
    input_format = [0, 3, 6]
    # output data is the moving distance and its direction
    output_format = [0, 2]

    random_shift, shuffle, transforms = 0, False, []

    if mode == 'train':
        random_shift = global_step_size // 2
        shuffle = True
        transforms.append(RandomHoriRotateSeq(input_format, output_format))
        global_step_size = kwargs.get('step_size')
        transforms = ComposeTransform(transforms)
        seq_type = GlobSequence
        global_window_size = kwargs.get('window_size')
        dataset = SequenceToSequenceDataset(seq_type, root_dir, data_list, global_step_size,
                                            global_window_size, random_shift = random_shift,
                                            transform = transforms, shuffle = shuffle)
    elif mode == 'val':
        shuffle = True
        global_step_size = kwargs.get('step_size')
        transforms = ComposeTransform(transforms)
        seq_type = GlobSequence
        global_window_size = kwargs.get('window_size')
        dataset = SequenceToSequenceDataset(seq_type, root_dir, data_list, global_step_size,
                                            global_window_size, random_shift = random_shift,
                                            transform = transforms, shuffle = shuffle)
    elif mode == 'val_test':
        shuffle = False
        global_step_size = kwargs.get('test_step_size')
        transforms = ComposeTransform(transforms)
        seq_type = GlobSequence
        global_window_size = kwargs.get('window_size')
        dataset = SequenceToSequenceDataset(seq_type, root_dir, data_list, global_step_size,
                                            global_window_size, random_shift = random_shift,
                                            transform = transforms, shuffle = shuffle)
    elif mode == 'test':
        shuffle = False
        global_step_size = kwargs.get('test_step_size')
        transforms = ComposeTransform(transforms)
        seq_type = GlobSequenceTest
        global_window_size = kwargs.get('window_size')
        dataset = SequenceToSequenceDatasetTest(seq_type, root_dir, data_list, global_step_size,
                                                global_window_size, random_shift = random_shift,
                                                transform = transforms, shuffle = shuffle)

    return dataset

def read_dir(dir_path):
    # read dirs from dir_path
    for _, dirs, _ in os.walk(dir_path):
        return dirs

def get_train_dataset(root_dir, **kwargs):
    trainlist = read_dir(root_dir)
    return get_dataset(root_dir, trainlist, mode = 'train', **kwargs)

def get_valid_dataset(root_dir, **kwargs):
    validlist = read_dir(root_dir)
    return get_dataset(root_dir, validlist, mode = 'val', **kwargs)

def get_valid_test_dataset(root_dir, dir, **kwargs):
    return get_dataset(root_dir, dir, mode = 'val_test', **kwargs)

def get_test_dataset(root_dir, dir, **kwargs):
    return get_dataset(root_dir, dir, mode = 'test', **kwargs)

### 4. Criterion

In [25]:
import json
import os
import sys
import time
import random
import argparse
from os import path as osp
from pathlib import Path

import numpy as np
import torch

class GlobalPosLoss(torch.nn.Module):
    def __init__(self):
        """
        Calculate position loss in global coordinate frame
        Target :- Global Velocity
        Prediction :- Global Velocity
        """
        super(GlobalPosLoss, self).__init__()
        self.mse_loss = torch.nn.MSELoss(reduction = 'none')

    def forward(self, pred, targ):
        # dts = 1 / 200
        dts = 1
        pred = pred * dts
        targ = targ * dts
        gt_pos = torch.cumsum(targ[:, 1:, ], 1)
        pred_pos = torch.cumsum(pred[:, 1:, ], 1)
        loss = self.mse_loss(pred_pos, gt_pos)
        # calculate the sum of absolute trajectory error
        return torch.mean(loss)

class MSEAverage():
    def __init__(self):
        self.count = 0
        self.targets = []
        self.predictions = []
        self.average = []

    def add(self, pred, targ):
        self.targets.append(targ)
        self.predictions.append(pred)
        self.average.append(np.average((pred - targ) ** 2, axis=(0, 1)))
        # print("The shape of average is: ", np.array(self.average).shape)
        # print("THe shape of np.average(np.array(self.average), axis=0) is: ", np.average(np.array(self.average), axis=0).shape)
        self.count += 1

    def get_channel_avg(self):
        average = np.average(np.array(self.average), axis=0)
        return average

    def get_total_avg(self):
        average = np.average(np.array(self.average), axis=0)
        return np.average(average)

    def get_elements(self, axis):
        return np.concatenate(self.predictions, axis=axis), np.concatenate(self.targets, axis=axis)

def reconstruct_traj(vector, **kwargs):
    global_sampling_rate = kwargs.get('sampling_rate', None)
    # reconstruct the vector to one sequence
    # velocity_sequence = vector.reshape(len(vector) * global_window_size, global_output_channel)

    velocity_sequence = vector * 1 / global_sampling_rate
    glob_pos = np.cumsum(velocity_sequence, axis = 0)

    return glob_pos

def compute_absolute_trajectory_error(pred, gt):
    """
    The Absolute Trajectory Error (ATE) defined in:
    A Benchmark for the evaluation of RGB-D SLAM Systems
    http://ais.informatik.uni-freiburg.de/publications/papers/sturm12iros.pdf

    Args:
        est: estimated trajectory
        gt: ground truth trajectory. It must have the same shape as est.

    Return:
        Absolution trajectory error, which is the Root Mean Squared Error between
        two trajectories.
    """
    return np.sqrt(np.mean((pred - gt) ** 2))


def compute_relative_trajectory_error(est, gt, delta, max_delta = -1):
    """
    The Relative Trajectory Error (RTE) defined in:
    A Benchmark for the evaluation of RGB-D SLAM Systems
    http://ais.informatik.uni-freiburg.de/publications/papers/sturm12iros.pdf

    Args:
        est: the estimated trajectory
        gt: the ground truth trajectory.
        delta: fixed window size. If set to -1, the average of all RTE up to max_delta will be computed.
        max_delta: maximum delta. If -1 is provided, it will be set to the length of trajectories.

    Returns:
        Relative trajectory error. This is the mean value under different delta.
    """
    if max_delta == -1:
        max_delta = est.shape[0]
    # print("delta: ", delta)
    deltas = np.array([min(delta, max_delta - 1)])
    # deltas = np.array([delta]) if delta > 0 else np.arange(1, min(est.shape[0], max_delta))
    rtes = np.zeros(deltas.shape[0])
    for i in range(deltas.shape[0]):
        # For each delta, the RTE is computed as the RMSE of endpoint drifts from fixed windows
        # slided through the trajectory.
        err = est[deltas[i]:] + gt[:-deltas[i]] - est[:-deltas[i]] - gt[deltas[i]:]
        rtes[i] = np.sqrt(np.mean(err ** 2))

    # The average of RTE of all window sized is returned.
    rtes = rtes[~np.isnan(rtes)]
    return np.mean(rtes)

def compute_position_drift_error(pos_pred, pos_gt):
    """
    Params:
        pos_pred: predicted position [seq_len, 2]
        pos_gt: ground truth position [seq_len, 2]
    """
    position_drift = np.linalg.norm((pos_gt[-1] - pos_pred[-1]))
    delta_position = pos_gt[1:] - pos_gt[:-1]
    delta_length = np.linalg.norm(delta_position, axis=1)
    moving_len = np.sum(delta_length)

    return position_drift / moving_len

def compute_distance_error(pos_pred, pos_gt):
    """
    Params:
        pos_pred: predicted position [seq_len, 2]
        pos_gt: ground truth position [seq_len, 2]
    """
    distance_error = np.linalg.norm((pos_gt - pos_pred), axis=1)

    return distance_error

def compute_heading_error(preds, targets):
    """
    Params:
        pos_pred: predicted position [seq_len, 2]
        pos_gt: ground truth position [seq_len, 2]
    """
    # Find the index of preds with zero norm
    zero_norm_index = np.where(np.linalg.norm(preds, axis=1) == 0)[0]
    # Remove the zero norm index
    preds = np.delete(preds, zero_norm_index, axis=0)
    targets = np.delete(targets, zero_norm_index, axis=0)
    # Find the index of targets with zero norm
    zero_norm_index = np.where(np.linalg.norm(targets, axis=1) == 0)[0]
    # Remove the zero norm index
    preds = np.delete(preds, zero_norm_index, axis=0)
    targets = np.delete(targets, zero_norm_index, axis=0)

    pred_v = np.linalg.norm(preds, axis=1)
    targ_v = np.linalg.norm(targets, axis=1)

    pred_o = preds / pred_v[:, np.newaxis]
    targ_o = targets / targ_v[:, np.newaxis]

    # calculate the heading angle of the predicted and target vectors
    pred_heading = np.arctan2(pred_o[:, 1], pred_o[:, 0])
    targ_heading = np.arctan2(targ_o[:, 1], targ_o[:, 0])

    # calculate the heading error
    heading_error = np.mean(np.abs(pred_heading - targ_heading))
    # convert to degrees
    heading_error = heading_error * 180 / np.pi

    return heading_error

### 5. Utils

In [47]:
import os
import numpy as np
import matplotlib.pyplot as plt
import os.path as osp

def format_string(*argv, sep=' '):
    result = ''
    for val in argv:
        if isinstance(val, (tuple, list, np.ndarray)):
            for v in val:
                result += format_string(v, sep=sep) + sep
        else:
            result += str(val) + sep
    return result[:-1]

def draw_trajectory(pos_pred, pos_gt, dir_name, ate, rte, **kwargs):
    """
    :param data:
    :pos_pred: (N, 2)
    :pos_gt: (N, 2)
    :dir_name: test directory
    :ate: average trajectory error
    :rte: relative trajectory error
    """
    global_out_dir = kwargs.get('out_dir', None)
    if global_out_dir is None:
        raise ValueError('out_dir is needed')

    plt.figure(figsize=(8, 5), dpi = 400)
    plt.plot(pos_pred[:, 0], pos_pred[:, 1], label = 'Predicted')
    plt.plot(pos_gt[:, 0], pos_gt[:, 1], label = 'Ground truth')
    plt.title(dir_name)
    print("make title success")
    # Show words in latex format
    plt.xlabel('$m$')
    plt.ylabel('$m$')
    plt.axis('equal')
    plt.legend()
    plt.title('ATE:{:.3f}, RTE:{:.3f}'.format(ate, rte), y = 0, loc = 'right')
    plt.show()
    plt.savefig(osp.join(global_out_dir, '{}.png'.format(dir_name)))

pos_pred= (N, 2)
pos_gt= (N, 2)
dir_name= '/kaggle/working/test_results'
draw_trajectory(pos_pred, pos_gt, dir_name, ate, rte, out_dir='output_directory')


NameError: name 'N' is not defined

### 6. Main Function

In [27]:
import os
import time
import argparse
from os import path as osp
from pathlib import Path
from tqdm import tqdm
import numpy as np
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader

def get_model(mode, **kwargs):
    global_input_channel = kwargs.get('input_channel')
    global_output_channel = kwargs.get('output_channel')
    global_dropout = kwargs.get('dropout')
    global_batch_size = kwargs.get('batch_size')
    global_test_batch_size = kwargs.get('test_batch_size')
    global_device = kwargs.get('device')
    global_layers = kwargs.get('layers')
    global_layer_size = kwargs.get('layer_size')

    if mode == 'train':
        print("LSTM model")
        network = BilinearLSTMSeqNetwork(global_input_channel, global_output_channel, global_batch_size, global_device,
                        lstm_size = global_layer_size, lstm_layers = global_layers, dropout = global_dropout).to(global_device)
    elif mode == 'test':
        print("LSTM model")
        network = BilinearLSTMSeqNetwork(global_input_channel, global_output_channel, global_test_batch_size, global_device,
                        lstm_size = global_layer_size, lstm_layers = global_layers, dropout = global_dropout).to(global_device)
    try:
        pytorch_total_params = sum(p.numel() for p in network.parameters() if p.requires_grad)
    except:
        pytorch_total_params = 0
    print('Network constructed. trainable parameters: {}'.format(pytorch_total_params))
    return network

def train(**kwargs):
    # load config
    global_data_dir = kwargs.get('data_dir')
    global_val_data_dir = kwargs.get('val_data_dir')
    global_batch_size = kwargs.get('batch_size')
    global_epochs = kwargs.get('epochs')
    global_num_workers = kwargs.get('num_workers')
    global_device = kwargs.get('device')
    global_out_dir = kwargs.get('out_dir', None)
    global_learning_rate = kwargs.get('learning_rate')
    global_save_interval = kwargs.get('save_interval')
    global_sampling_rate = kwargs.get('sampling_rate')
    # Loading data
    start_t = time.time()
    train_dataset = get_train_dataset(global_data_dir, **kwargs)
    val_dataset = get_valid_dataset(global_val_data_dir, **kwargs)
    train_loader = DataLoader(train_dataset, batch_size = global_batch_size, num_workers = global_num_workers, shuffle = True,
                              drop_last = True)
    val_loader = DataLoader(val_dataset, batch_size = global_batch_size, shuffle = True, drop_last = True)
    end_t = time.time()
    print('Training and validation set loaded. Time usage: {:.3f}s'.format(end_t - start_t))
    # read val for sequence test
    test_dirs = read_dir(global_val_data_dir)



    global device
    device = torch.device(global_device if torch.cuda.is_available() else 'cpu')
    print("Device: {}".format(device))

    if global_out_dir:
        if not osp.isdir(global_out_dir):
            os.makedirs(global_out_dir)
        if not osp.isdir(osp.join(global_out_dir, 'checkpoints')):
            os.makedirs(osp.join(global_out_dir, 'checkpoints'))

    print('\nNumber of train samples: {}'.format(len(train_dataset)))
    train_mini_batches = len(train_loader)
    if val_dataset:
        print('Number of val samples: {}'.format(len(val_dataset)))
        val_mini_batches = len(val_loader)

    network = get_model('train', **kwargs).to(device)
    testnetwork = get_model('test', **kwargs).to(device)
    criterion = GlobalPosLoss()

    optimizer = torch.optim.Adam(network.parameters(), global_learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience = 10, factor = 0.75, verbose = True, eps = 1e-12)
    quiet_mode = kwargs.get('quiet', False)
    use_scheduler = kwargs.get('use_scheduler', True)

    start_epoch = 0
    step = 0
    best_val_loss = np.inf
    train_errs = np.zeros(global_epochs)

    print("Starting from epoch {}".format(start_epoch))
    try:
        for epoch in range(start_epoch, global_epochs):
            log_line = ''
            network.train()
            train_vel = MSEAverage()
            train_loss = 0
            start_t = time.time()

            for bid, batch in enumerate(tqdm(train_loader)):
                feat, targ, _, _ = batch
                feat, targ = feat.to(device), targ.to(device)
                optimizer.zero_grad()
                predicted = network(feat)
                train_vel.add(predicted.cpu().detach().numpy(), targ.cpu().detach().numpy())
                loss = criterion(predicted, targ)
                train_loss += loss.cpu().detach().numpy()
                loss.backward()
                optimizer.step()
                step += 1

            train_errs[epoch] = train_loss / train_mini_batches
            end_t = time.time()
            if not quiet_mode:
                print('-' * 25)
                print('Epoch {}, time usage: {:.3f}s, loss: {}, vec_loss {}/{:.6f}'.format(
                    epoch, end_t - start_t, train_errs[epoch], train_vel.get_channel_avg(), train_vel.get_total_avg()))

            saved_model = False
            if val_loader:
                network.eval()
                val_vel = MSEAverage()
                val_loss = 0
                for bid, batch in enumerate(val_loader):
                    feat, targ, _, _ = batch
                    feat, targ = feat.to(device), targ.to(device)
                    optimizer.zero_grad()
                    pred = network(feat)
                    val_vel.add(pred.cpu().detach().numpy(), targ.cpu().detach().numpy())
                    val_loss += criterion(pred, targ).cpu().detach().numpy()
                val_loss = val_loss / val_mini_batches

                if not quiet_mode:
                    print('Validation loss: {} vec_loss: {}/{:.6f}'.format(val_loss, val_vel.get_channel_avg(),
                                                                                val_vel.get_total_avg()))

                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    saved_model = True
                    if global_out_dir:
                        model_path = osp.join(global_out_dir, 'checkpoints', 'checkpoint_%d.pt' % epoch)
                        torch.save({'model_state_dict': network.state_dict(),
                                    'epoch': epoch,
                                    'loss': train_errs[epoch],
                                    'optimizer_state_dict': optimizer.state_dict()}, model_path)
                        print('Best Validation Model saved to ' + model_path)
                if use_scheduler:
                    scheduler.step(val_loss)

            print("Reconstruct the validation trajectory and compute the ATE, RTE, PDE, and AYE")
            testnetwork.load_state_dict(network.state_dict())
            network.eval().to(device)
            ate_all, rte_all, pde_all = [], [], []
            aye_all = []
            # Every minute
            pred_per_min = global_sampling_rate * 60 # 2 + 0.5*(x - 1) = 60
            # Test for every sequence
            for i in range(len(test_dirs)):
            # for i in range(1):
                seq_dir = [test_dirs[i]]
                seq_dataset = get_valid_test_dataset(global_val_data_dir, seq_dir, **kwargs)
                feat, targ = seq_dataset.get_lstm_test_seq()

                feat = torch.from_numpy(feat).to(device)
                pred = testnetwork(feat).cpu().detach().numpy()

                pred = np.squeeze(pred, axis = 0)
                targ = np.squeeze(targ, axis = 0)
                # Reconstruct the trajectory
                pos_pred = reconstruct_traj(pred, **kwargs)
                pos_gt = reconstruct_traj(targ, **kwargs)

                # Compute the ATE and RTE
                ate = compute_absolute_trajectory_error(pos_pred, pos_gt)
                rte = compute_relative_trajectory_error(pos_pred, pos_gt, delta = pred_per_min)
                pde = compute_position_drift_error(pos_pred, pos_gt)
                heading_error = compute_heading_error(pred, targ)
                ate_all.append(ate)
                if rte >= 0:
                    rte_all.append(rte)
                pde_all.append(pde)
                aye_all.append(heading_error)

            ate_all = np.array(ate_all)
            rte_all = np.array(rte_all)
            pde_all = np.array(pde_all)
            aye_all = np.array(aye_all)

            measure = format_string('ATE', 'RTE', 'PDE', 'AYE',  sep = '\t')
            values = format_string(np.mean(ate_all), np.mean(rte_all), np.mean(pde_all), np.mean(aye_all), sep = '\t')
            print(measure + '\n' + values)

            if global_out_dir and not saved_model and (epoch + 1) % global_save_interval == 0:  # save even with validation
                model_path = osp.join(global_out_dir, 'checkpoints', 'icheckpoint_%d.pt' % epoch)
                torch.save({'ate': np .mean(ate_all),
                            'rte': np.mean(rte_all),
                            'pde': np.mean(pde_all),
                            'aye': np.mean(aye_all),}, model_path)
                torch.save({'model_state_dict': network.state_dict(),
                            'epoch': epoch,
                            'loss': train_errs[epoch],
                            'optimizer_state_dict': optimizer.state_dict()}, model_path)
                print('Model saved to ' + model_path)

            if np.isnan(train_loss):
                print("Invalid value. Stopping training.")
                break
    except KeyboardInterrupt:
        print('-' * 60)
        print('Early terminate')

    print('Training completed')
    if global_out_dir:
        model_path = osp.join(global_out_dir, 'checkpoints', 'checkpoint_latest.pt')
        torch.save({'model_state_dict': network.state_dict(),
                    'epoch': epoch,
                    'optimizer_state_dict': optimizer.state_dict()}, model_path)

# Test and Save the Trajectory for Both Seen and Unseen Data
seen_unseen_dataset = {'id1': 'tracermini_hw101_test20230311112635T',
                'id2': 'tracermini_hw101_test20230311111507T',
                'id3': 'tracermini_hw101_test20230311112235T',
                'id4': 'tracermini_hw101_test20230313011357T',
                'id5': 'tracermini_hw101_test20230311111842T',
                'id6': 'tracermini_hw101_test20230313010954T',
                'id7': 'tracermini_hw101_test20230313010546T',
                'id8': 'tracermini_hw101_test20230313011731T',
                'id9': 'tracermini_hw101_test20230313013335T',
                'id10': 'tracermini_hw101_test20230311111027T',
                'id11': 'tracermini_hw101_test20230313012956T',
                'id12': 'tracermini_hw101_test20230313010204T',
                'id13': 'tracermini_unseen_hw520230314091844T',
                'id14': 'tracermini_unseen_cym20230314101001T',
                'id15': 'tracermini_unseen_hw520230314082319T',
                'id16': 'tracermini_unseen_cym20230314101636T',
                'id17': 'tracermini_unseen_hw520230314083031T',
                'id18': 'tracermini_unseen_hw520230314091110T',
                'id19': 'tracermini_unseen_cym20230314100816T',
                'id20': 'tracermini_unseen_cym20230314101230T',
                'id21': 'tracermini_unseen_cym20230314100559T',
                'id22': 'tracermini_unseen_hw520230314081212T',
                'id23': 'tracermini_unseen_hw520230314082000T',
                'id24': 'tracermini_unseen_hw520230314091603T',
                'id25': 'tracermini_unseen_cym20230314100325T',
                'id26': 'tracermini_unseen_cym20230314101434T',
                'id27': 'tracermini_unseen_cym20230314100103T',
                'id28': 'tracermini_unseen_hw520230314091338T',
                'id29': 'tracermini_unseen_cym20230314101010T',
                'id30': 'tracermini_unseen_cym20230314101850T',
                'id31': 'tracermini_unseen_hw520230314081542T',
                'id32': 'tracermini_unseen_hw520230314090739T'}
seen_unseen_pred = {'id1': [],
            'id2': [],
            'id3': [],
            'id4': [],
            'id5': [],
            'id6': [],
            'id7': [],
            'id8': [],
            'id9': [],
            'id10': [],
            'id11': [],
            'id12': [],
            'id13': [],
            'id14': [],
            'id15': [],
            'id16': [],
            'id17': [],
            'id18': [],
            'id19': [],
            'id20': [],
            'id21': [],
            'id22': [],
            'id23': [],
            'id24': [],
            'id25': [],
            'id26': [],
            'id27': [],
            'id28': [],
            'id29': [],
            'id30': [],
            'id31': [],
            'id32': []}
def test_lstm(**kwargs):
    # load config
    global_dataset = kwargs.get('dataset')
    global_model_type = kwargs.get('model_type')
    global_num_workers = kwargs.get('num_workers')
    global_sampling_rate = kwargs.get('sampling_rate')
    global_device = kwargs.get('device')
    global_out_dir = kwargs.get('out_dir', None)
    global_test_dir = kwargs.get('test_dir', None)
    global_out_dir = kwargs.get('out_dir', None)
    global_model_path = kwargs.get('model_path', None)

    global device
    device = torch.device(global_device if torch.cuda.is_available() else 'cpu')

    if global_test_dir is None:
        raise ValueError('Test_path is needed.')

    # read dirs
    test_dirs = read_dir(global_test_dir)

    # Make sure the test output dir exists
    if global_out_dir and not osp.exists(global_out_dir):
        os.makedirs(global_out_dir)
    # Load the model config
    if global_model_path is None:
        raise ValueError('Model path is needed.')

    checkpoint = torch.load(global_model_path, map_location=global_device)

    network = get_model('test', **kwargs)
    network.load_state_dict(checkpoint.get('model_state_dict'))
    # network.load_state_dict(checkpoint.get('model_state_dict'))
    print("The model is loaded.")
    network.eval().to(device)
    print('Model {} loaded to device {}.'.format(global_model_path, device))

    # Test for every sequence
    for i in range(len(test_dirs)):
    # for i in range(1):
        seq_dir = [test_dirs[i]]
        seq_dataset = get_test_dataset(global_test_dir, seq_dir, **kwargs)
        feat = seq_dataset.get_lstm_test_seq()

        feat = torch.from_numpy(feat).to(device)
        pred = network(feat).cpu().detach().numpy()

        pred = np.squeeze(pred, axis = 0)

        # Reconstruct the trajectory
        print("Reconstruct the {}".format(seq_dir[0]))
        pos_pred = reconstruct_traj(pred, **kwargs)

        # Find the index of the sequence
        for key, value in seen_unseen_dataset.items():
            if value == seq_dir[0]:
                index = key
                seen_unseen_pred[index] = pos_pred

        # # Make directory
        # if global_out_dir and not osp.exists(osp.join(global_out_dir, seq_dir[0])):
        #     os.makedirs(osp.join(global_out_dir, seq_dir[0]))
        # # Save the trajectory
        # np.save(osp.join(global_out_dir, seq_dir[0], 'pred.npy'), pos_pred)

def test(**kwargs):
    # Model type
    print("Testing LSTM model")
    test_lstm(**kwargs)

### 7. Training

In [28]:
# Load config settings
kwargs = load_config()

import warnings
# Suspend warnings
warnings.filterwarnings('ignore')
train(**kwargs)

Training and validation set loaded. Time usage: 12.639s
Device: cuda:0

Number of train samples: 18634
Number of val samples: 3270
LSTM model
Network constructed. trainable parameters: 216620
LSTM model
Network constructed. trainable parameters: 216620
Starting from epoch 0


100%|██████████| 258/258 [00:10<00:00, 24.81it/s]


-------------------------
Epoch 0, time usage: 10.402s, loss: 6231.685725722202, vec_loss [0.27687997 0.34201962]/0.309450
Validation loss: 4251.38990342882 vec_loss: [0.31316587 0.35118878]/0.332177
Best Validation Model saved to /kaggle/working/prediction_model/checkpoints/checkpoint_0.pt
Reconstruct the validation trajectory and compute the ATE, RTE, PDE, and AYE
ATE	RTE	PDE	AYE
17.489653	9.011737216602672	0.32183152	119.31425731404524


100%|██████████| 258/258 [00:10<00:00, 25.17it/s]


-------------------------
Epoch 1, time usage: 10.253s, loss: 3582.7976869095205, vec_loss [0.27719176 0.48369846]/0.380445
Validation loss: 3534.950244140625 vec_loss: [0.25240335 0.4703616 ]/0.361382
Best Validation Model saved to /kaggle/working/prediction_model/checkpoints/checkpoint_1.pt
Reconstruct the validation trajectory and compute the ATE, RTE, PDE, and AYE
ATE	RTE	PDE	AYE
22.017792	10.387772560119629	0.39470088	111.90974423655649


...

-------------------------
Epoch 147, time usage: 10.125s, loss: 799.1540478846823, vec_loss [0.09876986 0.13082145]/0.114796
Validation loss: 927.8447258843316 vec_loss: [0.08890598 0.11321028]/0.101058
Reconstruct the validation trajectory and compute the ATE, RTE, PDE, and AYE
ATE	RTE	PDE	AYE
6.792725	3.1362285072153266	0.1317988	64.1180798146037


100%|██████████| 258/258 [00:10<00:00, 25.14it/s]


-------------------------
Epoch 148, time usage: 10.265s, loss: 813.160144184911, vec_loss [0.1025702 0.1257487]/0.114159
Validation loss: 910.3170545789931 vec_loss: [0.10174907 0.11627926]/0.109014
Reconstruct the validation trajectory and compute the ATE, RTE, PDE, and AYE
ATE	RTE	PDE	AYE
3.629222	2.330399578267878	0.06567859	68.27602142725665


100%|██████████| 258/258 [00:10<00:00, 25.41it/s]


-------------------------
Epoch 149, time usage: 10.158s, loss: 762.8375784703927, vec_loss [0.10284672 0.12330511]/0.113076
Validation loss: 1031.197664388021 vec_loss: [0.10543503 0.12984084]/0.117638
Reconstruct the validation trajectory and compute the ATE, RTE, PDE, and AYE
ATE	RTE	PDE	AYE
6.535742	3.350755648179488	0.11837521	70.05320187742183
Training completed


In [29]:
# Check Saved Checkpoints
# Define the directory path
dir_path = '/kaggle/working/prediction_model/checkpoints'

# Loop over all files in the directory
for filename in os.listdir(dir_path):
    # Check if the file is a regular file (not a directory)
    if os.path.isfile(os.path.join(dir_path, filename)):
        # Do something with the file
        print(filename)

checkpoint_3.pt
checkpoint_24.pt
checkpoint_1.pt
checkpoint_5.pt
checkpoint_44.pt
checkpoint_92.pt
icheckpoint_39.pt
icheckpoint_99.pt
checkpoint_138.pt
checkpoint_4.pt
checkpoint_54.pt
checkpoint_50.pt
icheckpoint_19.pt
checkpoint_38.pt
checkpoint_13.pt
checkpoint_146.pt
checkpoint_79.pt
checkpoint_31.pt
checkpoint_83.pt
checkpoint_11.pt
checkpoint_30.pt
checkpoint_16.pt
checkpoint_64.pt
checkpoint_59.pt
checkpoint_40.pt
checkpoint_53.pt
checkpoint_65.pt
checkpoint_61.pt
checkpoint_10.pt
checkpoint_111.pt
checkpoint_39.pt
checkpoint_2.pt
checkpoint_69.pt
checkpoint_82.pt
checkpoint_73.pt
checkpoint_26.pt
icheckpoint_119.pt
checkpoint_94.pt
checkpoint_22.pt
checkpoint_106.pt
checkpoint_99.pt
checkpoint_0.pt
icheckpoint_79.pt
checkpoint_81.pt
checkpoint_85.pt
checkpoint_76.pt
checkpoint_23.pt
checkpoint_42.pt
checkpoint_latest.pt
checkpoint_7.pt
checkpoint_33.pt
icheckpoint_59.pt
checkpoint_17.pt
icheckpoint_139.pt
checkpoint_105.pt


Download the checkpoints.

In [30]:
!zip -r checkpoints.zip /kaggle/working/prediction_model

  adding: kaggle/working/prediction_model/ (stored 0%)
  adding: kaggle/working/prediction_model/checkpoints/ (stored 0%)
  adding: kaggle/working/prediction_model/checkpoints/checkpoint_3.pt (deflated 8%)
  adding: kaggle/working/prediction_model/checkpoints/checkpoint_24.pt (deflated 7%)
  adding: kaggle/working/prediction_model/checkpoints/checkpoint_1.pt (deflated 8%)
  adding: kaggle/working/prediction_model/checkpoints/checkpoint_5.pt (deflated 8%)
  adding: kaggle/working/prediction_model/checkpoints/checkpoint_44.pt (deflated 8%)
  adding: kaggle/working/prediction_model/checkpoints/checkpoint_92.pt (deflated 7%)
  adding: kaggle/working/prediction_model/checkpoints/icheckpoint_39.pt (deflated 7%)
  adding: kaggle/working/prediction_model/checkpoints/icheckpoint_99.pt (deflated 7%)
  adding: kaggle/working/prediction_model/checkpoints/checkpoint_138.pt (deflated 8%)
  adding: kaggle/working/prediction_model/checkpoints/checkpoint_4.pt (deflated 7%)
  adding: kaggle/working/pred

In [31]:
from IPython.display import FileLink
FileLink(r'checkpoints.zip')

/kaggle/working/checkpoints.zip

### 8. Testing

#### 8.1 Test for Seen Dataset

In [33]:
# Please change the output dir & model path
TEST_DIR = '/kaggle/input/comp7310-project-1-imu-indoor-tracking/original_data/test_seen' # Dataset directory for testing (unseen_subjects_test_set)
OUT_DIR = '/kaggle/working/test_results/test_seen' # Output directory for both traning and testing
MODEL_PATH = '/kaggle/working/prediction_model/checkpoints/checkpoint_105.pt' # Model path for testing
# Load config settings
kwargs = load_config()

import warnings
# Suspend warnings
warnings.filterwarnings('ignore')
test(**kwargs)

Testing LSTM model
LSTM model
Network constructed. trainable parameters: 216620
The model is loaded.
Model /kaggle/working/prediction_model/checkpoints/checkpoint_105.pt loaded to device cuda:0.
Reconstruct the tracermini_hw101_test20230313011731T
Reconstruct the tracermini_hw101_test20230311111842T
Reconstruct the tracermini_hw101_test20230311111507T
Reconstruct the tracermini_hw101_test20230311112635T
Reconstruct the tracermini_hw101_test20230313012956T
Reconstruct the tracermini_hw101_test20230313010954T
Reconstruct the tracermini_hw101_test20230313010204T
Reconstruct the tracermini_hw101_test20230311111027T
Reconstruct the tracermini_hw101_test20230313010546T
Reconstruct the tracermini_hw101_test20230311112235T
Reconstruct the tracermini_hw101_test20230313011357T
Reconstruct the tracermini_hw101_test20230313013335T


#### 8.2 Test for Unseen Dataset

In [35]:
# Please change the output dir & model path
TEST_DIR = '/kaggle/input/comp7310-project-1-imu-indoor-tracking/original_data/test_unseen' # Dataset directory for testing (unseen_subjects_test_set)
OUT_DIR = '/kaggle/working/test_results/test_unseen' # Output directory for both traning and testing
MODEL_PATH = '/kaggle/working/prediction_model/checkpoints/checkpoint_105.pt' # Model path for testing
# Load config settings
kwargs = load_config()
import warnings
# Suspend warnings
warnings.filterwarnings('ignore')
test(**kwargs)

Testing LSTM model
LSTM model
Network constructed. trainable parameters: 216620
The model is loaded.
Model /kaggle/working/prediction_model/checkpoints/checkpoint_105.pt loaded to device cuda:0.
Reconstruct the tracermini_unseen_cym20230314101010T
Reconstruct the tracermini_unseen_hw520230314081212T
Reconstruct the tracermini_unseen_cym20230314101001T
Reconstruct the tracermini_unseen_hw520230314091603T
Reconstruct the tracermini_unseen_hw520230314083031T
Reconstruct the tracermini_unseen_cym20230314101850T
Reconstruct the tracermini_unseen_cym20230314101230T
Reconstruct the tracermini_unseen_cym20230314100103T
Reconstruct the tracermini_unseen_hw520230314090739T
Reconstruct the tracermini_unseen_cym20230314100325T
Reconstruct the tracermini_unseen_hw520230314082000T
Reconstruct the tracermini_unseen_cym20230314100816T
Reconstruct the tracermini_unseen_hw520230314091844T
Reconstruct the tracermini_unseen_hw520230314091338T
Reconstruct the tracermini_unseen_hw520230314081542T
Reconstruc

In [38]:
# Load the checkpoint
checkpoint_path = '/kaggle/working/prediction_model/checkpoints/checkpoint_105.pt'
checkpoint = torch.load(checkpoint_path, map_location='cpu')
# Load ATE，RTE，PDE, and AYE values from the checkpoint
ate = checkpoint.get('ate',None)
rte = checkpoint.get('rte',None)
pde = checkpoint.get('pde',None)  
aye = checkpoint.get('aye',None)

#### 8.3 Check the Results

In [39]:
for key, value in seen_unseen_pred.items():
    # print the length of each sequence
    print(seen_unseen_dataset[key], len(value))

tracermini_hw101_test20230311112635T 42097
tracermini_hw101_test20230311111507T 37913
tracermini_hw101_test20230311112235T 43462
tracermini_hw101_test20230313011357T 38324
tracermini_hw101_test20230311111842T 42839
tracermini_hw101_test20230313010954T 39610
tracermini_hw101_test20230313010546T 38378
tracermini_hw101_test20230313011731T 37610
tracermini_hw101_test20230313013335T 38254
tracermini_hw101_test20230311111027T 44186
tracermini_hw101_test20230313012956T 38694
tracermini_hw101_test20230313010204T 39085
tracermini_unseen_hw520230314091844T 19545
tracermini_unseen_cym20230314101001T 283
tracermini_unseen_hw520230314082319T 31657
tracermini_unseen_cym20230314101636T 22690
tracermini_unseen_hw520230314083031T 21208
tracermini_unseen_hw520230314091110T 25327
tracermini_unseen_cym20230314100816T 18914
tracermini_unseen_cym20230314101230T 21364
tracermini_unseen_cym20230314100559T 24483
tracermini_unseen_hw520230314081212T 37662
tracermini_unseen_hw520230314082000T 34311
tracermini_un

#### 8.4 Save the Results

In [40]:
with open("submission.csv", "w") as f:
    # The first row must be "Id, Category"
    f.write("Id,Prediction\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for key, value in seen_unseen_pred.items():
        # print the length of each sequence
        print(seen_unseen_dataset[key], len(value))
        # print the prediction
        for i in range(len(value)):
            f.write("{},{}\n".format(key+'_'+str(i), value[i]))

tracermini_hw101_test20230311112635T 42097
tracermini_hw101_test20230311111507T 37913
tracermini_hw101_test20230311112235T 43462
tracermini_hw101_test20230313011357T 38324
tracermini_hw101_test20230311111842T 42839
tracermini_hw101_test20230313010954T 39610
tracermini_hw101_test20230313010546T 38378
tracermini_hw101_test20230313011731T 37610
tracermini_hw101_test20230313013335T 38254
tracermini_hw101_test20230311111027T 44186
tracermini_hw101_test20230313012956T 38694
tracermini_hw101_test20230313010204T 39085
tracermini_unseen_hw520230314091844T 19545
tracermini_unseen_cym20230314101001T 283
tracermini_unseen_hw520230314082319T 31657
tracermini_unseen_cym20230314101636T 22690
tracermini_unseen_hw520230314083031T 21208
tracermini_unseen_hw520230314091110T 25327
tracermini_unseen_cym20230314100816T 18914
tracermini_unseen_cym20230314101230T 21364
tracermini_unseen_cym20230314100559T 24483
tracermini_unseen_hw520230314081212T 37662
tracermini_unseen_hw520230314082000T 34311
tracermini_un

In [48]:
#zip
import os
import zipfile
import datetime

def file2zip(packagePath, zipPath):
    '''
  :param packagePath: 文件夹路径
  :param zipPath: 压缩包路径
  :return:
  '''
    zip = zipfile.ZipFile(zipPath, 'w', zipfile.ZIP_DEFLATED)
    for path, dirNames, fileNames in os.walk(packagePath):
        fpath = path.replace(packagePath, '')
        for name in fileNames:
            fullName = os.path.join(path, name)
            name = fpath + '\\' + name
            zip.write(fullName, name)
    zip.close()


if __name__ == "__main__":
    # 文件夹路径
    packagePath = '/kaggle/working/'
    zipPath = '/kaggle/working/output.zip'
    if os.path.exists(zipPath):
        os.remove(zipPath)
    file2zip(packagePath, zipPath)
    print("打包完成")
    print(datetime.datetime.utcnow())


打包完成
2023-10-27 14:16:36.116966
